In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd

In [2]:
## read in url csv
urls = pd.read_csv('rfs_free_article_url.csv')
urls.head()


,url
0,/rfs/article/22/3/925/1578888
1,/rfs/article/23/4/1503/1592566
2,/rfs/article/24/8/2864/1574102
3,/rfs/article/24/9/3187/1571750
4,/rfs/article/24/11/3688/1589538


In [3]:
 def get_html(url):
    try:
        kv = {'user-agent': 'Mozilla/5.0'}
        url = "https://academic.oup.com" + url
        print(url)

        r = requests.get(url, headers=kv, timeout=30)
        r.raise_for_status()
        r.status_code
        r.encoding = r.apparent_encoding
        return r

    except:
        return

In [27]:
def get_hypothesis(soup):

    hypotheses = []
    # case 1:
    results = soup.find_all(name='div', attrs='statement')

    for result in results:
        content = result.text
        # print(content)
        match = re.search('hypothes', content, re.I)
        if match:
            hypotheses.append(content)

    # case 2:
    results = soup.find_all(name='p', attrs='chapter-para')

    for result in results:
        content = result.text
        # print(content)
        try:
            res = re.search('we.{1,10}hypothesize that.{1,300}\.', content, re.I)
            if res:
                hypotheses.append(res.group(0))
        except:
            pass

    # case 3:
    for result in results:
        content = result.text
        # print(content)
        try:
            res = re.search('hypothesis \d[a-z]?[\s:].{1,300}', content, re.I)
            if res:
                hypotheses.append(res.group(0))
        except:
            pass

    # case 4:
    for result in results:
        # print(result)
        content = result.text
        # print(content)
        try:
            res = re.search(r'Our.{1,10}hypothesis (\bis\b|\bwas\b) that.{1,300}\.', content, re.I)
            if res:
                hypotheses.append(res.group(0))
        except:
            pass

    # case 5:
    for result in results:
        # print(result)
        content = result.text
        # print(content)
        try:
            res = re.search(r'we.{5,15}the hypothesis that.{1,300}\.', content, re.I)
            if res:
                hypotheses.append(res.group(0))
        except:
            pass

    # case 6:
    for result in results:
        # print(result)
        content = result.text
        # print(content)
        try:
            res = re.search(r'.{100,400}\. To test this hypothesis', content)
            if res:
                hypotheses.append(res.group(0)[:-24]) # remove "To test this hypothesis"

            res = re.search(r'.{100,400}\. To test these hypotheses', content)
            if res:
                hypotheses.append(res.group(0)[:-24]) # remove "To test this hypothesis"
        except:
            pass


    return hypotheses

In [28]:
def main(url):
    response = get_html(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return get_hypothesis(soup)

## test extraction function

In [29]:
# test case 1
print(main("/rfs/article/31/4/1418/3896050"))

https://academic.oup.com/rfs/article/31/4/1418/3896050
['Hypothesis 1.Firm performance increases in the size of the industry tournament prize, as measured by the external pay gap.', 'Hypothesis 2.To the extent that the trailing CEO can affect risk, then the stock return volatility and riskiness of investment and financial policies of the aspirant’s firm increase in the size of the prize.', 'Hypothesis 3.A higher probability of the aspirant winning the tournament and capturing the gain increases the incentive effect of the tournament on the performance, risk, and policy of the aspirant’s firm.', 'We hypothesize that these tournament incentive effects appear in data from the managerial labor market. In the time series, the effects of variation in the size of the tournament prize on effort and observable firm performance should apply to the aspirant CEO.', 'We hypothesize that aspects of financial and investment policy that increase firm risk, such as R&D intensity, firm focus, and book l

In [30]:
# test case 2
print(main("/rfs/article/30/7/2229/3058111"))


https://academic.oup.com/rfs/article/30/7/2229/3058111
['We hypothesize that VESTING is positively associated with the revision in analyst forecasts over the same quarter.', 'We thus hypothesize that vesting equity increases the frequency with which the manager narrowly beats the forecast at announcement.', 'Our hypothesis is that |$\\beta < 0$|\u2060. The control variables CONTROLS are as discussed in Section 1.3.']


In [31]:
# test case 3
print(main("/rfs/article/29/4/863/1896505"))


https://academic.oup.com/rfs/article/29/4/863/1896505
['Hypothesis 1: Investors holding equity pay more attention to their investments in rising stock markets than in falling markets.', 'Hypothesis 1: Investors holding equity pay more attention to their investments in rising stock markets than in falling markets.', 'Hypothesis 2: Attention decreases in market volatility.Hypothesis 3: Attention increases in news media coverage of the stock market.', 'Hypothesis 2: Attention decreases in market volatility.', 'Hypothesis 3: Attention increases in news media coverage of the stock market.', 'Hypothesis 4: Patterns in aggregate attention and conditional trading differ due to differing underlying economic drivers.', 'Hypothesis 4: Patterns in aggregate attention and conditional trading differ due to differing underlying economic drivers.', 'Hypothesis 5: The average level of attention and ostrich effect behavior are greater for older (closer to retirement) investors and for wealthier investor

In [32]:
# test case 4
print(main("/rfs/article/32/5/1854/5427780"))

https://academic.oup.com/rfs/article/32/5/1854/5427780
['Our main hypothesis is that the FinTech lending model represents a technological innovation that reduces frictions in mortgage lending, such as lengthy loan processing, capacity constraints, inefficient refinancing, and limited access to finance by some borrowers.']


In [33]:
# test case 5
print(main("/rfs/article/32/11/4117/5315531"))

https://academic.oup.com/rfs/article/32/11/4117/5315531
['we examine the hypothesis that expropriation through SOVs can destroy shareholder value. In poorly governed firms, managers may find it easier to extract resources for their own gain.']


In [34]:
# test case 6
print(main("/rfs/article/32/9/3366/5298322"))

https://academic.oup.com/rfs/article/32/9/3366/5298322
['Finally, if weakly capitalized GIIPS banks strongly increased their loan supply to low-quality GIIPS borrowers at advantageous interest rates due to either government moral suasion or a clearance of a credit supply backlog, we would expect a credit increase especially of domestic banks to domestic low-IC ratio firms. ']


## get hypothesis from all free articles

In [36]:
results = {}
for url in urls['url'].values:
    print("trying", url)
    hypotheses = main(url)
    print('{:} hypotheses found\n'.format(len(hypotheses)))
    if hypotheses:
        results[url] = hypotheses
    

trying /rfs/article/22/3/925/1578888
https://academic.oup.com/rfs/article/22/3/925/1578888
0 hypotheses found

trying /rfs/article/23/4/1503/1592566
https://academic.oup.com/rfs/article/23/4/1503/1592566
1 hypotheses found

trying /rfs/article/24/8/2864/1574102
https://academic.oup.com/rfs/article/24/8/2864/1574102
0 hypotheses found

trying /rfs/article/24/9/3187/1571750
https://academic.oup.com/rfs/article/24/9/3187/1571750
0 hypotheses found

trying /rfs/article/24/11/3688/1589538
https://academic.oup.com/rfs/article/24/11/3688/1589538
1 hypotheses found

trying /rfs/article/27/9/2539/1614366
https://academic.oup.com/rfs/article/27/9/2539/1614366
0 hypotheses found

trying /rfs/article/27/10/2841/1607080
https://academic.oup.com/rfs/article/27/10/2841/1607080
4 hypotheses found

trying /rfs/article/27/11/3099/1597777
https://academic.oup.com/rfs/article/27/11/3099/1597777
1 hypotheses found

trying /rfs/article/27/12/3699/1575316
https://academic.oup.com/rfs/article/27/12/3699/15753

In [73]:
import pandas as pd
df = pd.DataFrame({'url': list(results.keys()), 'hypotheses': list(results.values())})

In [74]:
df.head()

,url,hypotheses
0,/rfs/article/23/4/1503/1592566,[we test the hypothesis that the index is outp...
1,/rfs/article/24/11/3688/1589538,[we hypothesize that the strength of the macro...
2,/rfs/article/27/10/2841/1607080,[Our hypothesis is that investors' marginal ut...
3,/rfs/article/27/11/3099/1597777,[We can reject the hypothesis that the average...
4,/rfs/article/27/12/3389/1575014,[Given that sophisticated investors are likely...


In [78]:
# drop duplicates
df['hypotheses'] = df['hypotheses'].apply(lambda x: list(set(x)))
df['num_hypotheses'] = df['hypotheses'].apply(len)

In [80]:
df['num_hypotheses'].sum()

72

In [81]:
df.to_csv('extracted_hypothesis.csv', index=False)